# `get_rec_item` Function

If the recs are item clusters and you want to get actual items, use this!

In [13]:
import numpy as np
import pandas as pd

from random_gen import *
from get_rec import *

def get_rec_item(df_rec, top_k, ic_assignment):
    
    """Returns the top K item recommendations for each user in the user list
    
            Parameters:
                    df_rec (pandas.DataFrame): Table containing the top N item cluster recommendations for each user in the user list
                    ic_assignment (array-like): List containing the cluster assignment of each item
                    top_n (int): Number of items to recommend

            Returns:
                    df_rec_item (pandas.DataFrame): Table containing the top K item recommendations for each user in the user list
                    
    """

    # Create recommendation table
    df_rec_item = pd.DataFrame()
    df_rec_item['user_id'] = df_rec['user_id']
    
    for i in range(top_k):
        df_rec['rank_'+str(i+1)] = np.zeros(df_rec_item.shape[0])
        for j in range(df_rec_item.shape[0]):
            
            
            if uc_assignment is None:
                df_rec.iloc[j, i+1] = cluster_rec[user_list[j], top_n-i-1]
            else:
                df_rec.iloc[j, i+1] = cluster_rec[uc_assignment[user_list[j]], top_n-i-1]
                
    return df_rec_item

## Example

In [14]:
n_user = 100
n_item = 50
sample_size = 10
n_user_cluster = 5
n_item_cluster = 5
top_n = 3
random_seed = 1

user_id_list = list(range(n_user))
user_list = random_user_list(n_user, sample_size, random_seed)
uc_assignment = random_user_cluster(n_user, n_user_cluster, random_seed)
utility_matrix_o, utility_matrix = random_utility_matrix(n_user_cluster, n_item_cluster, random_seed)
df_rec = get_rec(utility_matrix, utility_matrix_o, user_list, top_n, uc_assignment)
df_rec

,user_id,rank_1,rank_2,rank_3
0,80,1.0,2.0,4.0
1,84,3.0,4.0,1.0
2,33,3.0,4.0,1.0
3,81,0.0,3.0,1.0
4,93,3.0,1.0,0.0
5,17,0.0,3.0,1.0
6,36,3.0,1.0,0.0
7,82,1.0,2.0,4.0
8,69,4.0,2.0,0.0
9,65,3.0,1.0,0.0


In [15]:
ic_assignment = random_user_cluster(n_item, n_item_cluster, random_seed=2)
ic_assignment

array([0, 0, 3, 2, 3, 0, 2, 1, 3, 2, 4, 4, 4, 3, 4, 2, 3, 3, 2, 1, 2, 4,
       3, 0, 4, 3, 1, 2, 0, 4, 4, 2, 4, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 1,
       1, 1, 4, 2, 3, 0])

In [19]:
top_k = 10
get_rec_item(df_rec, top_k, ic_assignment)

,user_id
0,80
1,84
2,33
3,81
4,93
5,17
6,36
7,82
8,69
9,65
